基於 4chan 魔改版 NovelAILeaks (naifu) 製作。 [來源](https://boards.4channel.org/g/thread/89095460#p89097704)

使用官方前端 + 優化版後端，可突破75限制，支持所有模型。

In [1]:
!ls -lR

.:
total 4
drwxr-xr-x 1 root root 4096 Dec 20 20:19 sample_data

./sample_data:
total 55504
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Dec 20 20:19 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Dec 20 20:19 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Dec 20 20:19 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Dec 20 20:19 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


In [4]:
# Load the Drive helper and mount
from google.colab import drive
 
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from PIL import Image
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from google.colab.patches import cv2_imshow

transform1 = transforms.Compose([transforms.Resize([22, 22]), transforms.ToTensor()])


class FaceLandmarkDataset(Dataset):
    def __len__(self) -> int:
        print("!!!!!!!!", len(self.landmarks_frame))
        return len(self.landmarks_frame)

    def __init__(self, csv_file: str, root_dir: str, transform=None) -> None:
        super().__init__()
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self, index: int):
        img_name = self.landmarks_frame.Filename[index]
        print(img_name)
        img_path = os.path.join('/content/drive/MyDrive/tmp/', str(img_name) + ".png")
        print(img_path)
        # with Image.open(img_path) as img: 
        #     image = img.convert('RGB') 
        img = cv2.imread(img_path)

        PIL_image = Image.fromarray(img)  # 这里ndarray_image为原来的numpy数组类型的输入

        img1 = transform1(PIL_image)
        img2 = img1.numpy() * 255
        img2 = img2.astype('uint8')
        img2 = np.transpose(img2, (1, 2, 0))
        img2 = np.reshape(img2, (3, 22,22))
        landmarks = self.landmarks_frame.values[index, 1:].astype('float')
        print(landmarks)
        # landmarks = np.reshape(landmarks,newshape=(-1,2)) 
        # if self.transform is not None: 
        #     image = self.transform(image) 

        cv2_imshow(img2)
        # print(str(img2.size()))
        return img2, landmarks

    ########################以上为数据读取类（返回：image,landmarks）###############################


trans = transforms.Compose(transforms=[transforms.RandomResizedCrop(size=128),
                                       transforms.ToTensor()])

face_dataset = FaceLandmarkDataset(csv_file=r'/content/drive/MyDrive/tmp/Run History.csv',
                                   root_dir='/content/drive/MyDrive/tmp', transform=trans)
loader = DataLoader(dataset=face_dataset,
                    batch_size=4,
                    shuffle=True,
                    num_workers=4)

# img, label=face_dataset[1]
# cv2_imshow(img.numpy().transpose(1,2,0)*255)
# print(label)

# dataiterator = iter(loader)
# img, label = dataiterator.next()

# print(img.size(), label.size())

FileNotFoundError: ignored

In [7]:
#@title ### 0. 檢查 GPU 工作狀態

!nvidia-smi

Mon Jan  2 12:47:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title ### 1. 下載 Novel AI API 後端、模型 
#@markdown 如果下載速度太慢可以嘗試 restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

/content
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libc-ares2
The following NEW packages will be installed:
  aria2 libc-ares2
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,274 kB of archives.
After this operation, 4,912 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.14.0-1ubuntu0.1_amd64.deb ...
Unpacking libc-ares2:amd64 (1.14.0-1ubuntu0.1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.33.1-1_amd64.deb ...
Unpacking aria2 (1.33.1-1) ...
Setting up libc-ares2:amd64 (1.14.0-1ubuntu0.1) ...
Setting up aria2 (1.33.1-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-

In [ ]:
#@title ### 2. 安裝依賴
#@markdown 耐心等待安裝完成

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

/content/naifu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 34.4 MB/s 
     |████████████████████████████████| 468 kB 75.1 MB/s 
+ virtualenv venv
created virtual environment CPython3.7.15.final.0-64 in 868ms
  creator CPython3Posix(dest=/content/naifu/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3, setuptools==65.5.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
+ . venv/bin/activate
++ '[' venv/bin/activate = ./setup.sh ']'
++ deactivate nondestructive
++ unset -f pydoc
++ '[' -z '' ']'
++ '[' -z '' ']'
++ hash -r
++ '[' -z '' ']'
++ unset VIRTUAL_ENV
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/naifu/venv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 33.0M  100 33.0M    0     0  45.0M      0 --:--:-- --:--:-- --:--:-- 45.0M


In [ ]:
#@title ### 3. 啟動模型
#@markdown 訪問輸出的映射地址（以 `trycloudflare.com` / `bore.pub` 結尾）即可。
#@markdown - 請等待模型加載完成（出現`Application startup complete`字樣）再訪問
#@markdown - cloudflare 提供的服務偶爾會出現請求超時，可換用 bore 隧道

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

/content/naifu
2023-01-02T13:44:44Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2023-01-02T13:44:44Z INF Requesting new quick Tunnel on trycloudflare.com...
2023-01-02T13:44:44.470154Z  INFO bore_cli::client: connected to server remote_port=44247
2023-01-02T13:44:44.470195Z  INFO bore_cli::client: listening at bore.pub:44247
2023-01-02T13:44:46Z INF +--------------------------------------------------------------------------------------------+
2023-01-02T13:44:46Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2023-01-02T13:44:46Z INF |  https://cape-postcard-consumers-assuming.trycloudflare.com             

In [ ]:
#@title ### 4. （可選）使用 7G 的 `animefull-latest` 模型運行
#@markdown 默認使用的是 4G 大小的 animefull-final-pruned 模型。如果想使用 7G 的 animefull-latest 模型，運行這個

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969